In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.optim import Adam
from sklearn.metrics import mean_absolute_error, mean_squared_error,mean_absolute_percentage_error
from sklearn.preprocessing import StandardScaler
import torch_geometric
from torch_geometric.data import Data
from torch_geometric.nn import SAGEConv, GATConv, GeneralConv, TransformerConv
from torch.nn import GRU


In [2]:

class MLP(nn.Module):
    def __init__(self, in_channels, hidden_layers, out_channels):
        super().__init__()
        layers = []
        prev_layer = in_channels

        for hidden_dim in hidden_layers:
            layers.append(nn.Linear(prev_layer, hidden_dim))
            layers.append(nn.ReLU())
            prev_layer = hidden_dim

        layers.append(nn.Linear(prev_layer, out_channels))
        self.mlp = nn.Sequential(*layers)

    def forward(self, x):
        return self.mlp(x)


class GRUDecoder(nn.Module):
    def __init__(self, in_channels, out_channels, hidden_gru=64):
        super().__init__()
        hidden_layers = [128, 64, 32]

        self.gru = nn.GRU(in_channels, hidden_gru, batch_first=True)
        self.mlp = MLP(hidden_gru, hidden_layers, out_channels)

    def forward(self, z):
        gru_out, _ = self.gru(z)
        output = self.mlp(gru_out.squeeze())
        return output


class GNNEncoder1(torch.nn.Module):
    def __init__(self, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = SAGEConv((-1, -1), hidden_channels)
        self.conv2 = SAGEConv((-1, -1), out_channels)

    def forward(self, x, edge_index,edge_attr=None):
        x = self.conv1(x, edge_index).relu()
        x = self.conv2(x, edge_index)
        return x

class GNNEncoder2(torch.nn.Module):
    def __init__(self, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GATConv((-1, -1), hidden_channels, add_self_loops=False)
        self.conv2 = GATConv((-1, -1), out_channels, add_self_loops=False)

    def forward(self, x, edge_index, edge_attr):
        x = self.conv1(x, edge_index, edge_attr).relu()
        x = self.conv2(x, edge_index, edge_attr)
        return x

class GNNEncoder3(torch.nn.Module):
    def __init__(self, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GeneralConv((-1, -1), hidden_channels, in_edge_channels=-1)
        self.conv2 = GeneralConv((-1, -1), out_channels, in_edge_channels=-1)

    def forward(self, x, edge_index, edge_attr):
        x = self.conv1(x, edge_index, edge_attr).relu()
        x = self.conv2(x, edge_index, edge_attr)
        return x

class GNNEncoder4(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, heads=1, dropout=0.0, edge_dim=None):
        super().__init__()
        self.conv1 = TransformerConv(
            in_channels=in_channels,
            out_channels=hidden_channels,
            heads=heads,
            concat=True,
            beta=False,
            dropout=dropout,
            edge_dim=edge_dim,
            bias=True,
            root_weight=True
        )
        self.conv2 = TransformerConv(
            in_channels=hidden_channels * heads,
            out_channels=out_channels,
            heads=heads,
            concat=True,
            beta=False,
            dropout=dropout,
            edge_dim=edge_dim,
            bias=True,
            root_weight=True
        )

    def forward(self, x, edge_index, edge_attr=None):
        x = self.conv1(x, edge_index, edge_attr).relu()
        x = self.conv2(x, edge_index, edge_attr)
        return x


In [3]:
class Model(torch.nn.Module):
    def __init__(self, input_size, hidden_channels, out_channels, edge_index, edge_attr=None, encoder_type='GNNEncoder4'):
        super().__init__()
        if encoder_type == 'GNNEncoder1':
            self.encoder = GNNEncoder1(hidden_channels, hidden_channels)
        elif encoder_type == 'GNNEncoder2':
            self.encoder = GNNEncoder2(hidden_channels, hidden_channels)
        elif encoder_type == 'GNNEncoder3':
            self.encoder = GNNEncoder3(hidden_channels, hidden_channels)
        elif encoder_type == 'GNNEncoder4':
            self.encoder = GNNEncoder4(input_size, hidden_channels, hidden_channels, edge_dim=edge_attr.size(1) if edge_attr is not None else None)
        else:
            raise ValueError(f"Unknown encoder type: {encoder_type}")

        self.decoder = GRUDecoder(hidden_channels, out_channels)
        self.edge_index = edge_index
        self.edge_attr = edge_attr

    def forward(self, x):
        z = self.encoder(x, self.edge_index, self.edge_attr)
        z = z.unsqueeze(1)
        return self.decoder(z)

In [4]:
try:
    speed_data = pd.read_csv('dataset/PeMSD7_V_228.csv', header=None)
    adjacency_matrix = pd.read_csv('dataset/PeMSD7_W_228.csv', header=None)
except FileNotFoundError:
    import os
    import requests
    import zipfile

    url = "https://github.com/VeritasYin/STGCN_IJCAI-18/raw/master/dataset/PeMSD7_Full.zip"
    os.makedirs('dataset', exist_ok=True)
    response = requests.get(url)
    zip_path = 'dataset/PeMSD7_Full.zip'
    with open(zip_path, 'wb') as f:
        f.write(response.content)

    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall('dataset')

    speed_data = pd.read_csv('dataset/PeMSD7_V_228.csv', header=None)
    adjacency_matrix = pd.read_csv('dataset/PeMSD7_W_228.csv', header=None)


In [5]:
scaler = StandardScaler()
speed_data = scaler.fit_transform(speed_data)

train_size = int(len(speed_data) * 0.8)
train_data = speed_data[:train_size]
test_data = speed_data[train_size:]


In [6]:
train_data = torch.tensor(train_data, dtype=torch.float32)
test_data = torch.tensor(test_data, dtype=torch.float32)

# Create edge index from adjacency matrix
adjacency_matrix = adjacency_matrix.values  # Convert to numpy array
edge_index = torch.tensor(np.stack(np.nonzero(adjacency_matrix)), dtype=torch.long)
edge_attr = adjacency_matrix[edge_index[0], edge_index[1]]
edge_attr = torch.tensor(edge_attr, dtype=torch.float).unsqueeze(1)  # Shape: [num_edges, 1]
# Create a graph object
graph = Data(edge_index=edge_index,edge_attr=edge_attr)

In [7]:
input_size = speed_data.shape[1]
hidden_channels = input_size
out_channels = speed_data.shape[1]
num_epochs = 50
learning_rate = 0.001

In [8]:
encoder_type = 'GNNEncoder1'

In [ ]:
model = Model(input_size, hidden_channels, out_channels, graph.edge_index, edge_attr=None, encoder_type=encoder_type)
optimizer = Adam(model.parameters(), lr=learning_rate)
loss_fn = nn.MSELoss()

In [13]:
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()

    # Prepare input and target
    input_data = train_data[:-1]
    target_data = train_data[1:]

    # Forward pass
    output = model(input_data)
    loss = loss_fn(output, target_data)

    # Backward pass
    loss.backward()
    optimizer.step()

    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item()}')

Epoch 1/50, Loss: 1.005180835723877
Epoch 2/50, Loss: 1.002534031867981
Epoch 3/50, Loss: 1.0002368688583374
Epoch 4/50, Loss: 0.9973888993263245
Epoch 5/50, Loss: 0.9938968420028687
Epoch 6/50, Loss: 0.9897443056106567
Epoch 7/50, Loss: 0.9848119020462036
Epoch 8/50, Loss: 0.9789284467697144
Epoch 9/50, Loss: 0.9722875356674194
Epoch 10/50, Loss: 0.9646264910697937
Epoch 11/50, Loss: 0.955955445766449
Epoch 12/50, Loss: 0.9462308287620544
Epoch 13/50, Loss: 0.9355654716491699
Epoch 14/50, Loss: 0.9241706728935242
Epoch 15/50, Loss: 0.9122286438941956
Epoch 16/50, Loss: 0.8998342752456665
Epoch 17/50, Loss: 0.8869591951370239
Epoch 18/50, Loss: 0.8734019994735718
Epoch 19/50, Loss: 0.8591899871826172
Epoch 20/50, Loss: 0.8444371223449707
Epoch 21/50, Loss: 0.829206109046936
Epoch 22/50, Loss: 0.8139382600784302
Epoch 23/50, Loss: 0.7984638214111328
Epoch 24/50, Loss: 0.7832241654396057
Epoch 25/50, Loss: 0.7677274942398071
Epoch 26/50, Loss: 0.7521836757659912
Epoch 27/50, Loss: 0.7375

In [14]:
model.eval()
with torch.no_grad():
    # Prepare input and target
    input_data = test_data[:-1]
    target_data = test_data[1:]

    # Forward pass
    output = model(input_data)

    # Calculate metrics
    mse = mean_squared_error(target_data.numpy(), output.numpy())
    mae = mean_absolute_error(target_data.numpy(), output.numpy())
    mape = mean_absolute_percentage_error(target_data.numpy(), output.numpy())
    rmse = np.sqrt(mse)

    print(f'MSE: {mse}')
    print(f'MAE: {mae}')
    print(f'MAPE: {mape}')
    print(f'RMSE: {rmse}')

MSE: 0.5069800019264221
MAE: 0.4565073847770691
MAPE: 2.3420441150665283
RMSE: 0.7120252818028459
